In [1]:
!git clone https://github.com/benedekrozemberczki/MUSAE.git

Cloning into 'MUSAE'...
remote: Enumerating objects: 672, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 672 (delta 2), reused 0 (delta 0), pack-reused 666
Receiving objects: 100% (672/672), 19.88 MiB | 20.99 MiB/s, done.
Resolving deltas: 100% (358/358), done.


In [4]:
import csv
import json
import networkx as nx
import numpy as np
import torch
from scipy.sparse import csr_matrix

In [7]:
dir = ["DE", "ENGB", "ES", "FR", "PTBR", "RU"]

# for dataset

for dataset in dir:

    output_npz = {}

    with open("MUSAE/input/edges/" + dataset + "_edges.csv") as f2:
        csvreader = csv.reader(f2)
        next(csvreader)
        dic = []
        for row in csvreader:
            scr = row[0]
            dst = row[1]
            dic.append([int(scr), int(dst)])

        G = nx.Graph(dic)
        edges = np.array(dic)

        output_npz['edge'] = edges

        # labels: use "mature" as label for now
        node_num = G.number_of_nodes()
        print(dataset)
        print("number of nodes", node_num)
        print("number of edges", G.number_of_edges())
        labels = [0] * node_num
    
        with open("MUSAE/input/target/" + dataset+ "_target.csv") as f3:
            csvreader = csv.reader(f3)
            next(csvreader)
            dic = []
            for row in csvreader:
                node = row[5]
                if row[2] == 'True':
                    label = 1
                elif row[2] == 'False':
                    label = 0
                else:
                    print("label wrong!!!!")
    
                labels[int(node)] = int(label)
    
        output_npz['node_class'] = labels
    
        # features
        row1 = []
        col1 = []
        val1 = []
        f = open("MUSAE/input/features/" + dataset + ".json")
        data = json.load(f)
        for node in data:
            for feats in data[node]:
                row1.append(int(node))
                col1.append(int(feats))
                val1.append(1)
        
        features = csr_matrix((val1, (row1, col1)), shape=(node_num, 3170))
    
        output_npz['node_feats'] = features
    
        # node list
        output_npz['node_list'] = torch.from_numpy(np.ones(node_num))
    
    np.savez("Twitch-" + dataset[:2] + ".npz", **output_npz, allow_pickle=True)
    
    test_load = np.load("Twitch-" + dataset[:2] + ".npz")

    

DE
number of nodes 9498
number of edges 153138
ENGB
number of nodes 7126
number of edges 35324
ES
number of nodes 4648
number of edges 59382
FR
number of nodes 6549
number of edges 112666
PTBR
number of nodes 1912
number of edges 31299
RU
number of nodes 4385
number of edges 37304
